In [6]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/jared/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /home/jared/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to
[nltk_data]   

[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package sinica_treebank is already up-to-date!
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package smultron is already up-to-date!
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package state_union is already up-to-date!
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     /home/jared/nltk_data...
[nltk_data]    |   Package 

True

## Reading File and creating DataFrame

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()


In [9]:
import pandas as pd
wiki_files = pd.read_csv('wiki_sample.csv')
wiki_dataframe = pd.DataFrame(wiki_files)
wiki_dataframe

FileNotFoundError: [Errno 2] No such file or directory: 'wiki_sample.csv'

## Preprocessing and Creating Vocab

In [10]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

In [8]:
vocab = {}
term_doc_index = {}

def applyFunction(row, lenTitle, docID):
    total_words = 0
    unique_words = 0
    heaps_law_dataset = list()
    filteredContent = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        total_words += 1
        token = lemmatizer.lemmatize(token).lower()
        if token not in stop_words and token.isalnum():
            filteredContent.append(token)
            if token not in vocab:
                unique_words += 1
                vocab[token] = 1
                term_doc_index[token] = docID
            else:
                vocab[token] += 1
            heaps_law_dataset.append((total_words, unique_words))
        if total_words % 100 == 0:
            slope, intercept = heaps_law(heaps_law_dataset, total_words - 10000, total_words)
            print(slope)
            if slope < 0.01: 
                break
    return filteredContent

In [18]:
wiki_dataframe_pickle['tokenized_content'][0:1] = wiki_dataframe_pickle.progress_apply(lambda row: applyFunction(row, len(row['title']), row['id']), axis=1)

  0%|          | 1/1286264 [00:00<138:38:54,  2.58it/s]/home/jared/anaconda3/envs/project1/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
  0%|          | 28/1286264 [00:00<5:29:05, 65.14it/s] 

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


  0%|          | 53/1286264 [00:00<3:52:51, 92.06it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

  0%|          | 76/1286264 [00:01<3:56:18, 90.71it/s]

...

  0%|          | 572/1286264 [00:05<4:07:59, 86.41it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


  0%|          | 580/1286264 [00:06<3:42:21, 96.37it/s]


nan
nan
nan
nan
nan
nan
nan


KeyboardInterrupt: 

In [20]:
wiki_dataframe_pickle[0:1]

,content,title,id
0,Morocco–Saudi Arabia relations\r\n\r\nMoroccan...,Morocco–Saudi Arabia relations,1


In [7]:
import pickle
wiki_dataframe.to_pickle('./wiki_dataframe')

NameError: name 'wiki_dataframe' is not defined

In [4]:
wiki_dataframe_pickle = pd.read_pickle('./wiki_sample.pkl')

In [7]:
wiki_dataframe_pickle

,content,title,id
0,Morocco–Saudi Arabia relations\r\n\r\nMoroccan...,Morocco–Saudi Arabia relations,1
1,Anthony United States Post Office\r\n\r\nThe A...,Anthony United States Post Office,2
2,Dumraon (Vidhan Sabha constituency)\r\n\r\nDum...,Dumraon Vidhan Sabha constituency,3
3,Chaker Khazaal\r\n\r\nChaker Khazaal (born 28 ...,Chaker Khazaal,4
4,Vicente Pascual Pastor\r\n\r\nVicente Pascual ...,Vicente Pascual Pastor,5
...,...,...,...
1286259,Toppr\r\n\r\nToppr is a product of Haygot Educ...,Toppr,1286260
1286260,Outline of education\r\n\r\nThe following outl...,Outline of education,1286261
1286261,Raell Padamsee\r\n\r\nRaell Padamsee is an Ind...,Raell Padamsee,1286262
1286262,Megan Callaway\r\n\r\nMegan Callaway is a foun...,Megan Callaway,1286263


In [ ]:
vocab = {}
term_doc_index = {}
for token in wiki_dataframe['tokenized_content']:
    if token not in vocab:
                vocab[token] = 1
                term_doc_index[token] = docID
            else:
                vocab[token] += 1

In [1]:
wiki_dataframe['tokenized_content'][0]

NameError: name 'wiki_dataframe' is not defined

## Heaps Law

In [13]:
import matplotlib.pyplot as plt
import math

def heaps_law(list_to_graph, lb, ub):
        x = list()
        y = list()
        
        for item in list_to_graph[lb:ub]:
            x.append(item[0])
            y.append(item[1])
        
        slope, intercept = np.polyfit(np.log(x), np.log(y), 1) 
        return (slope, intercept)
        
#         expected_x = list()
#         expected_y = list()
        
#         k = 20
#         for n in range(lb, ub):
#             expected_x.append(n)
#             expected_y.append(math.pow(n, 0.5) * k)
            
       
#         expected_slope, expected_intercept = np.polyfit(np.log(expected_x), np.log(expected_y), 1) 
        

#         plt.loglog(x, y)
#         plt.loglog(bfX,bfY)
#         plt.savefig("heaps_law.png")
#         return (expected_slope - slope, expected_intercept - intercept)
        
    

Matplotlib is building the font cache; this may take a moment.
